In [11]:
import torch
from datasets.car_evaluation import CarEvaluation
from binhd.embeddings import CategoricalEncoder
from binhd.datasets import BaseDataset
from torch.utils.data import DataLoader
from module.record_encoder import RecordEncoder
from binhd.classifiers import BinHD
import torchmetrics

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

Using cpu device


In [3]:
dataset = CarEvaluation()
dimension = 1000
num_levels = 100
batch_size = 1000

In [4]:
X = dataset.features[dataset.categorical_features]

categorical_encoder = CategoricalEncoder(dimension)
X = categorical_encoder.fit_transform(X)

In [5]:
from sklearn.model_selection import train_test_split


y = list(dataset.labels)
y = torch.tensor(y).squeeze().to(device)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)  

train_dataset = BaseDataset(X_train.values, y_train)
test_dataset = BaseDataset(X_test.values, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [6]:
record_encode = RecordEncoder(dimension, dataset.num_features, num_levels)
record_encode = record_encode.to(device)

In [10]:
with torch.no_grad():
    samples = torch.tensor(X.values).to(device)
    labels = torch.tensor(y).squeeze().to(device)

    X_hv = record_encode(samples)

/tmp/ipykernel_83230/401827750.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(y).squeeze().to(device)


In [7]:
model = BinHD(dimension, dataset.num_classes)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_hv, labels, test_size=0.3, random_state = 0)  

In [15]:
from sklearn.metrics import accuracy_score


with torch.no_grad():
    model.fit(X_train,y_train)
    predictions = model.predict(X_test)  
    acc = accuracy_score(predictions, y_test)
    print("BinHD: Accuracy = ", acc)

    model.fit_adapt(X_train,y_train)
    predictions = model.predict(X_test)  
    acc = accuracy_score(predictions, y_test)
    print("BinHD - Adapt: Accuracy = ", acc)

BinHD: Accuracy =  0.22736030828516376


fit:   0%|          | 0/30 [00:00<?, ?it/s]

fit: 100%|██████████| 30/30 [00:00<00:00, 34.19it/s]

BinHD - Adapt: Accuracy =  0.15606936416184972
